In [1]:
import cv2
import numpy as np

In [2]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [3]:
def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [4]:
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

In [5]:
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

In [6]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    return cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

In [7]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)


In [8]:

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [9]:
def process_frame(frame):
    gray = grayscale(frame)
    blur = gaussian_blur(gray, 5)
    edges = canny(blur, 50, 150)
    
    imshape = frame.shape
    vertices = np.array([[(0, imshape[0]), (450, 320), (490, 320), (imshape[1], imshape[0])]], dtype=np.int32)
    roi_edges = region_of_interest(edges, vertices)
    
    lines = hough_lines(roi_edges, 1, np.pi/180, 15, 40, 20)
    
    line_img = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)
    if lines is not None:
        draw_lines(line_img, lines)
    
    result = weighted_img(line_img, frame)
    return result

In [11]:
# Load video
cap = cv2.VideoCapture("C:/Users/gaurav/Videos/test_video.mp4")

In [17]:
# Load video
input_video = 'C:/Users/gaurav/Videos/test_video.mp4'
output_video = 'C:/Users/gaurav/Videos/output_video.mp4'

cap = cv2.VideoCapture(input_video)

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            result = process_frame(frame)
            out.write(result)
        else:
            break

    cap.release()
    out.release()
    print(f"Output video saved as {output_video}")

Output video saved as C:/Users/gaurav/Videos/output_video.mp4
